<a href="https://colab.research.google.com/github/hyeonsuk/ai/blob/master/lab/source/CNN/CNNs_190816.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN Algorithms

## 참고 소스
* [모두의 딥러닝_김성훈](https://github.com/hunkim/DeepLearningZeroToAll/blob/master/lab-11-1-mnist_cnn.py)
* [솔라리스의 인공지능 연구실](http://solarisailab.com/archives/2325)

## Algorithms 작동을 위한 설정

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls '/content/drive'

'My Drive'


## LeNet-5

![alt text](https://drive.google.com/uc?export=view&id=1Sdnwq8-hCsitzB3Sgv5Nx--iA4sSvrId)

In [0]:
!ls /content/drive/My\ Drive/Colab\ Notebooks/MNIST_data

t10k-images-idx3-ubyte.gz  train-images-idx3-ubyte.gz
t10k-labels-idx1-ubyte.gz  train-labels-idx1-ubyte.gz


In [0]:
import tensorflow as tf
import random
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data

tf.reset_default_graph()
tf.set_random_seed(777)

mnist = input_data.read_data_sets("/coneten/drive/My\ Drive/Colab\ Notebooks/MNIST_data/", one_hot=True)

Extracting /coneten/drive/My\ Drive/Colab\ Notebooks/MNIST_data/train-images-idx3-ubyte.gz
Extracting /coneten/drive/My\ Drive/Colab\ Notebooks/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /coneten/drive/My\ Drive/Colab\ Notebooks/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /coneten/drive/My\ Drive/Colab\ Notebooks/MNIST_data/t10k-labels-idx1-ubyte.gz


In [0]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [0]:
# 신경망 모델
def CNN_model(x):
  x_img = tf.reshape(x, [-1, 28, 28, 1])
  
  W_conv1 = tf.Variable(tf.truncated_normal(shape=[5, 5, 1, 6], stddev=5e-2))
  b_conv1 = tf.Variable(tf.constant(0.1, shape=[6]))
  h_conv1 = tf.nn.relu(tf.nn.conv2d(x_img, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1)
  
  h_pool1 = tf.nn.avg_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
  
  W_conv2 = tf.Variable(tf.truncated_normal(shape=[5, 5, 6, 16], stddev=5e-2))
  b_conv2 = tf.Variable(tf.constant(0.1, shape=[16]))
  h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='VALID') + b_conv2)
  
  h_pool2 = tf.nn.avg_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
  
  W_fc1 = tf.Variable(tf.truncated_normal(shape=[5*5*16, 120], stddev=5e-2))
  b_fc1 = tf.Variable(tf.constant(0.1, shape=[120]))
  
  h_pool2_flat = tf.reshape(h_pool2, [-1, 5*5*16])
  h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
  
  W_fc2 = tf.Variable(tf.truncated_normal(shape=[120, 10], stddev=5e-2))
  b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))
  logits = tf.matmul(h_fc1, W_fc2) + b_fc2
  y_pred = tf.nn.softmax(logits)
  
  return y_pred, logits

In [0]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)

In [0]:
y_pred, logits = CNN_model(x)

In [0]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [0]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  
  print('Learing started. It takes sometime.')
  for epoch in range(training_epochs):
    avg_loss = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
      batch_xs, batch_ys = mnist.train.next_batch(batch_size)
      feed_dict = {x: batch_xs, y: batch_ys}
      c, _ = sess.run([loss, optimizer], feed_dict=feed_dict)
      avg_loss += c / total_batch
      
    print('Epoch:', '%04d' % (epoch+1), 'loss=', '{:.9f}'.format(avg_loss))
    
  print('Learning Finished!')
  
  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  print('Accuracy:', sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels}))

Learing started. It takes sometime.
Epoch: 0001 loss= 0.451709552
Epoch: 0002 loss= 0.123537475
Epoch: 0003 loss= 0.081617070
Epoch: 0004 loss= 0.062947082
Epoch: 0005 loss= 0.052232240
Epoch: 0006 loss= 0.044300938
Epoch: 0007 loss= 0.037309552
Epoch: 0008 loss= 0.033703001
Epoch: 0009 loss= 0.029245998
Epoch: 0010 loss= 0.026123583
Epoch: 0011 loss= 0.022436618
Epoch: 0012 loss= 0.021890410
Epoch: 0013 loss= 0.017681270
Epoch: 0014 loss= 0.017341792
Epoch: 0015 loss= 0.014183784
Learning Finished!
Accuracy: 0.9889
